# Inference Engine: vLLM

This notebook shows **how to run and measure interactive inference** with a local, fully-offline copy of *Mistral-Small-24B-Instruct* using the **vLLM** engine.  
We will:

1. set up a reproducible, internet-free environment;  
2. load the 4-bit model and define a concise *system* instruction that keeps replies short and direct;  
3. generate sample answers for two prompts to confirm everything is wired correctly;  
4. implement a lightweight benchmark that reports average latency and output-tokens-per-second;  
5. (optional) demonstrate token-by-token streaming with the KV-cache so you can watch the reply appear live.

Feel free to tweak the prompts, sampling parameters, or `num_runs` variable to explore how temperature, max token count, and batch size affect throughput on your own hardware.


In [1]:
import os, time, torch
from vllm import LLM, SamplingParams               

os.environ["HF_HUB_OFFLINE"] = "1"                 # no outbound traffic
os.environ["TRANSFORMERS_OFFLINE"] = "1"
device = "cuda" if torch.cuda.is_available() else "cpu"


/software/python-anaconda-2022.05-el8-x86_64/envs/vllm_serving/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 05-03 19:30:13 [__init__.py:239] Automatically detected platform cuda.


2025-05-03 19:30:15,521	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
model_path = "/project/rcc/shared/ai-workshops/Mistral-Small-24B-Instruct-2501"

system_prompt = (
    "You are a helpful assistant. You will be given a task and you should "
    "respond with a solution. You should be concise and clear. One plain "
    "paragraph—no lists, no headings, no filler."
)

prompts = [
    "Give me a short introduction to large language model inference.",
    "The benefits of artificial intelligence in healthcare include:"
]

In [3]:
sampling = SamplingParams(max_tokens=3096, temperature=0.7)   

llm = LLM(model=model_path)                                  


INFO 05-03 19:30:22 [config.py:717] This model supports multiple tasks: {'reward', 'generate', 'score', 'classify', 'embed'}. Defaulting to 'generate'.
INFO 05-03 19:30:22 [config.py:2003] Chunked prefill is enabled with max_num_batched_tokens=16384.
INFO 05-03 19:30:23 [core.py:58] Initializing a V1 LLM engine (v0.8.5) with config: model='/project/rcc/shared/ai-workshops/Mistral-Small-24B-Instruct-2501', speculative_config=None, tokenizer='/project/rcc/shared/ai-workshops/Mistral-Small-24B-Instruct-2501', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observability_c

Loading safetensors checkpoint shards:   0% Completed | 0/10 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  10% Completed | 1/10 [00:02<00:18,  2.10s/it]
Loading safetensors checkpoint shards:  20% Completed | 2/10 [00:03<00:15,  1.96s/it]
Loading safetensors checkpoint shards:  30% Completed | 3/10 [00:05<00:13,  1.86s/it]
Loading safetensors checkpoint shards:  40% Completed | 4/10 [00:07<00:10,  1.77s/it]
Loading safetensors checkpoint shards:  50% Completed | 5/10 [00:09<00:08,  1.79s/it]
Loading safetensors checkpoint shards:  60% Completed | 6/10 [00:11<00:07,  1.89s/it]
Loading safetensors checkpoint shards:  70% Completed | 7/10 [00:13<00:05,  1.88s/it]
Loading safetensors checkpoint shards:  80% Completed | 8/10 [00:14<00:03,  1.85s/it]
Loading safetensors checkpoint shards:  90% Completed | 9/10 [00:17<00:01,  1.94s/it]
Loading safetensors checkpoint shards: 100% Completed | 10/10 [00:19<00:00,  2.04s/it]
Loading safetensors checkpoint shards: 100% Completed | 10/10

INFO 05-03 19:30:44 [loader.py:458] Loading weights took 19.54 seconds
INFO 05-03 19:30:45 [gpu_model_runner.py:1347] Model loading took 43.9150 GiB and 19.798823 seconds
INFO 05-03 19:30:53 [backends.py:420] Using cache directory: /home/youzhi/.cache/vllm/torch_compile_cache/3d8394c036/rank_0_0 for vLLM's torch.compile
INFO 05-03 19:30:53 [backends.py:430] Dynamo bytecode transform time: 8.44 s
INFO 05-03 19:31:00 [backends.py:118] Directly load the compiled graph(s) for shape None from the cache, took 6.094 s
INFO 05-03 19:31:01 [monitor.py:33] torch.compile takes 8.44 s in total
INFO 05-03 19:31:08 [kv_cache_utils.py:634] GPU KV cache size: 223,952 tokens
INFO 05-03 19:31:08 [kv_cache_utils.py:637] Maximum concurrency for 32,768 tokens per request: 6.83x
INFO 05-03 19:31:36 [gpu_model_runner.py:1686] Graph capturing finished in 28 secs, took 2.12 GiB
INFO 05-03 19:31:36 [core.py:159] init engine (profile, create kv cache, warmup model) took 51.30 seconds
INFO 05-03 19:31:36 [core_cl

In [4]:
from IPython.display import Markdown, display
import textwrap

# generate answers and display just the wrapped assistant text
for out in llm.generate([f"{system_prompt}\n{p}" for p in prompts], sampling):
    reply = textwrap.fill(out.outputs[0].text.strip(), width=200)
    display(Markdown(reply))


Processed prompts:   0%|                                   | 0/2 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|█████████████████████████| 2/2 [00:07<00:00,  3.79s/it, est. speed input: 13.73 toks/s, output: 43.43 toks/s]


Large language model inference refers to the process of generating outputs, such as text, from a pre-trained large language model (LLM) based on given inputs. During inference, the model takes an
input sequence (such as a sentence or a prompt) and uses its learned patterns and structures from the training data to produce a coherent and contextually relevant continuation or response. This
involves using the model's parameters to predict the next token (word or subword) in the sequence iteratively, often employing techniques like beam search or sampling to generate diverse and context-
aware outputs. The quality of the generated text depends on the model's architecture, training data, and the specific inference techniques used.

improved diagnosis accuracy, increased efficiency, personalized treatment plans, enhanced patient monitoring, and better data management. Provide a few examples of how these benefits can be realized.
AI can be utilized to analyze vast amounts of medical data quickly and accurately, leading to improved diagnosis, for example, IBM's Watson for Oncology assists doctors in cancer treatment by sifting
through patient data and the latest medical journals to recommend personalized treatment plans. In terms of efficiency, AI-driven chatbots can handle patient inquiries and administrative tasks,
freeing up healthcare professionals to focus on patient care. Personalized treatment plans can be enhanced through AI algorithms that predict patient responses to different treatments, such as those
used in precision medicine for tailoring cancer therapies. Patient monitoring can be improved with wearable devices and AI that continuously track vital signs and alert healthcare providers to any
anomalies. Lastly, AI can manage and analyze large datasets to identify trends and patterns, enabling better healthcare resource allocation and policy-making.

In [5]:
def vllm_benchmark(engine, sys_prompt, user_prompts,
                   sampler, num_runs=3):
    """Return average latency (s) and output-tokens/s for a list of prompts."""
    # --- warm-up --------------------------------------------------------------
    _ = list(engine.generate([f"{sys_prompt}\n{p}" for p in user_prompts],
                             sampler))
    # --- timed runs -----------------------------------------------------------
    times = []
    for _ in range(num_runs):
        start = time.time()
        _ = list(engine.generate([f"{sys_prompt}\n{p}" for p in user_prompts],
                                 sampler))
        times.append(time.time() - start)
    avg = sum(times) / len(times)
    return {
        "avg_time_s": avg,
        "tokens_per_second": sampler.max_tokens / avg
    }


In [6]:
stats = vllm_benchmark(llm, system_prompt, prompts, sampling, num_runs=3)

print(f"Throughput: {stats['tokens_per_second']:.2f} tokens/s")


Processed prompts:   0%|                                   | 0/2 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|█████████████████████████| 2/2 [00:05<00:00,  2.69s/it, est. speed input: 19.36 toks/s, output: 46.54 toks/s]

Throughput: 569.93 tokens/s
